In [1]:
# rap 모델
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import random
import sys
import io
import re

Using TensorFlow backend.


In [0]:
lyric = pd.read_csv('./sample_data/music_lyrics_total.csv', encoding = 'utf-8')
child = pd.read_csv('./sample_data/music_lyrics_children_total.csv', encoding = 'utf-8')

In [3]:
child = child['lyrics'].drop_duplicates()
child.shape

(4209,)

In [4]:
text = ''

for i in range(2000):
    if child[i] != None:
        song = child[i]
#         print(song)
        song = re.sub('[\(\)-]', ' ', song)
        cleaned = song.lower().replace(' ', '\n')
    #     print(cleaned)
        text = text + " ".join(re.findall(r"[가-힣]+", cleaned)) + ' '
len(text)

315139

In [5]:
text2 = ''
for i in range(6000):
    if lyric['lyrics'][i] != None:
        song = lyric['lyrics'][i]
#         print(song)
        song = re.sub('[\(\)-]', ' ', song)
        cleaned = song.lower().replace(' ', '\n')
    #     print(cleaned)
        text2 = text2 + " ".join(re.findall(r"[가-힣]+", cleaned)) + ' '
len(text2)

2530914

In [6]:
text = text + text2
len(text)

2846053

In [7]:
tokens = re.findall(r"[a-zA-Z가-힣'\s]", text)

chars = sorted(list(set(tokens)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 1744


In [8]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('nb sequences:', len(sentences))

nb sequences: 955062


In [8]:
maxlen = 20
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('nb sequences:', len(sentences))

nb sequences: 948678


In [0]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [10]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
# model.add(Dropout(0.5)) # 추가
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))
model.summary()

Build model...





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               958976    
_________________________________________________________________
dense_1 (Dense)              (None, 1744)              224976    
_________________________________________________________________
activation_1 (Activation)    (None, 1744)              0         
Total params: 1,183,952
Trainable params: 1,183,952
Non-trainable params: 0
_________________________________________________________________


In [0]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.3, 0.5]: # 0.2, 0.5
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [12]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

history = model.fit(
    x, 
    y,
    batch_size=512,
    epochs=10,
    callbacks=[print_callback]
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





948678/948678 [==============================] - 433s 456us/step - loss: 3.3709
----- Generating text after Epoch: 0
----- diversity: 0.3
----- Generating with seed: "천생연분 배달의 민족과 심야식당남이사"
천생연분 배달의 민족과 심야식당남이사랑 나는 모두 다 함께 하고 싶어 그대 나의 마음을 기다려 내게 다시 돌아가네 내 맘을 내게 해 내 맘을 보여 우리 둘이 걷는 우리 둘만의 남자 나는 모두 다 같이 날 잡아 어디든 가 그래 나 봐 너와 함께 있어 나 그 때 그 사람 이제 내 맘 속에 나 나 나 나 나나 나나 나나 나나 나나나 나나 나나나 나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나 나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나나
----- diversity: 0.5
----- Generating with seed: "천생연분 배달의 민족과 심야식당남이사"
천생연분 배달의 민족과 심야식당남이사랑 보고 싶어 오늘 밤 나의 모습이 어디로 가 이제 이 순간 아직 너를 만나고 내 맘을 기다리는 너의 모두 다 또 불어오는 나의 맘을 만나게 미래 전부 다 시간을 보고 싶어 나 어디로 가야 해 내 옆에 있어 내가 있어 이제 내 맘을 너를 만난 게 네가 많은 걸 그렇게 있어 널 만나 봐 너무 미안해 오늘 밤 그 인사를 나를 않는 건

In [0]:
from keras.models import load_model

model.save('./sample_data/model_child2000_lyric6000_v1.h5') 